In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import random
import multiprocessing

from itertools import product
from datetime import date
from path import Path

import os
os.environ['NUMEXPR_MAX_THREADS'] = '32'

from mlbt.multiprocess import process_jobs
from mlbt.run_bt import run_bt
from mlbt.feature_eng import define_feature_configs
from mlbt.load_data import LENGTH_RANKING

DATA_DIR = Path("~/Dropbox/algotrading/data").expanduser()
s_csv = pd.read_csv(DATA_DIR / "symbols.csv", index_col="iqsymbol")

# Code to execute the pipeline inside a multiprocessing.Pool
# This creates the payloads that "run_frontend_payloads.ipynb" can use to generate the
# frontend payloads (.json loaded by the frontend)
ranking = ["equity_index", ""]


symbol_groups = list(
    {
        "agriculture": "Agriculture",
        "currency": "Currency",
        "energy": "Energy",
        "equity_index": "Equity Index",
        "interest_rate": "Interest Rate",
        "metals": "Metals",
    }.keys()
)

features = define_feature_configs()
print(len(features))
c_s = (len(features) // 32) + 1 
c_s = 1
print('c_s', c_s)
data_sim = {
    "start_date": [date(2000, 1, 1)],
    "end_date": [date(2021, 1, 1)],
    "symbol_groups": [symbol_groups],
#     "symbols": [['@NQ#C', '@TU#C']],
#     "symbols": [[x] for x in LENGTH_RANKING],
    "bar_type": ["dollar"],
    "binarize": ["fixed_horizon"],
#     "binarize_params": [6, 12],
#     "binarize_params": [6],
    "binarize_params": [6, 12, 50, 100, 200],
#     "binarize_params": [100, 200],
#     "binarize_params": [200],
#     "binarize_params": [6, 25, 50],
    "binarize_params": [25],
#     "binarize_params": [100],
#     "binarize_params": [6],
#     "binarize_params": [800],
    "alpha": ["none"],
#     "alpha": ["ma-cross_100_1000"],
#     "features": [features[i:i+c_s] for i in range(0, len(features), c_s)],
    "features": [features],
    "classifier": ["random_forest", "xgboost", "lgbm", "dummy"],
#     "classifier": ["xgboost"],
#     "classifier": ["dummy"],
#     "classifier": ["lgbm"],
    "num_threads": [32], # how many parallel hyperopti 
    "n_jobs": [32], # how many parallel infer
#     "num_threads": [8], # how many parallel hyperopti 
#     "n_jobs": [8], # how many parallel infer
#     "num_threads": [1], # how many parallel hyperopti 
#     "num_threads": [2], # how many parallel hyperopti 
    "feat_imp_n_estimators": [1000],
    "feat_imp_cv": [10],
    "func": [run_bt],
#     "feature_calc_only": [True],
#     "feature_imp_only": [True],
#     "check_completed": [True],
#     "optimize_hypers": [False],
    "reuse_hypers": [False],
#     "load_from_disk": [False],
#     "skip_feature_imp": [True],
#     "pca_transform": [False],
#     "standard_scale": [False],
#     "hypers_n_iter": [10],
}

33
c_s 1


C:\Users\doda\Anaconda3\envs\fincl2\lib\site-packages\sklearn\utils\deprecation.py:144: FutureWarning: The sklearn.ensemble.bagging module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)
C:\Users\doda\Anaconda3\envs\fincl2\lib\site-packages\sklearn\utils\deprecation.py:144: FutureWarning: The sklearn.ensemble.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)


In [3]:
datas = list(dict(zip(data_sim.keys(), x)) for x in product(*data_sim.values()))
datas
# for data in datas:
#     data['features'] = list(sorted(data['features'], key=lambda x:random.random()))


[{'start_date': datetime.date(2000, 1, 1),
  'end_date': datetime.date(2021, 1, 1),
  'symbol_groups': ['agriculture',
   'currency',
   'energy',
   'equity_index',
   'interest_rate',
   'metals'],
  'bar_type': 'dollar',
  'binarize': 'fixed_horizon',
  'binarize_params': 25,
  'alpha': 'none',
  'features': [{'name': 'log_ret'},
   {'name': 'close', 'symbol': 'VIX.XO'},
   {'name': 'ffd', 'd': 0.5},
   {'name': 'roll', 'window': 10},
   {'name': 'rollimp', 'window': 10},
   {'name': 'amihud', 'window': 10},
   {'name': 'kyle', 'window': 10},
   {'name': 'ewm_mean', 'window': 10, 'symbol': {'name': 'volratio'}},
   {'name': 'roll', 'window': 25},
   {'name': 'rollimp', 'window': 25},
   {'name': 'amihud', 'window': 25},
   {'name': 'kyle', 'window': 25},
   {'name': 'ewm_mean', 'window': 25, 'symbol': {'name': 'volratio'}},
   {'name': 'roll', 'window': 50},
   {'name': 'rollimp', 'window': 50},
   {'name': 'amihud', 'window': 50},
   {'name': 'kyle', 'window': 50},
   {'name': 'ewm

In [4]:
len(datas)

4

In [5]:
# [x['features'][0] for x in datas]

In [6]:
num_threads = max(1, min(multiprocessing.cpu_count(), len(datas)) - 0)
num_threads
      

4

In [7]:
# res = process_jobs(datas, num_threads=num_threads)

In [8]:
for data in datas:
    run_bt(**data)

2020-02-26 12:57:06,026 config: {'DATA_DIR': Path('C:\\Users\\doda/Dropbox/algotrading/data'), 'F_PAYLOAD_DIR': Path('C:\\Users\\doda/pr/fincl/frontend/public/payloads'), 'start_date': datetime.date(2000, 1, 1), 'end_date': datetime.date(2021, 1, 1), 'data_freq': 'minutely', 'downsampling': 'cusum', 'vol_estimate': 100, 'symbol_groups': ['agriculture', 'currency', 'energy', 'equity_index', 'interest_rate', 'metals'], 'features': [{'name': 'log_ret'}, {'name': 'close', 'symbol': 'VIX.XO'}, {'name': 'ffd', 'd': 0.5}, {'name': 'roll', 'window': 10}, {'name': 'rollimp', 'window': 10}, {'name': 'amihud', 'window': 10}, {'name': 'kyle', 'window': 10}, {'name': 'ewm_mean', 'window': 10, 'symbol': {'name': 'volratio'}}, {'name': 'roll', 'window': 25}, {'name': 'rollimp', 'window': 25}, {'name': 'amihud', 'window': 25}, {'name': 'kyle', 'window': 25}, {'name': 'ewm_mean', 'window': 25, 'symbol': {'name': 'volratio'}}, {'name': 'roll', 'window': 50}, {'name': 'rollimp', 'window': 50}, {'name': '

2020-02-26 12:57:12,304 XG#C: Feature engineering for 33 features
2020-02-26 12:57:12,669 Joined 33 features into (71217, 33) shape
2020-02-26 12:57:12,670 EX#C: Feature engineering for 33 features
2020-02-26 12:57:13,030 Joined 33 features into (74018, 33) shape
2020-02-26 12:57:13,031 @YM#C: Feature engineering for 33 features
2020-02-26 12:57:13,371 Joined 33 features into (39311, 33) shape
2020-02-26 12:57:13,372 @BO#C: Feature engineering for 33 features
2020-02-26 12:57:13,740 Joined 33 features into (93715, 33) shape
2020-02-26 12:57:13,741 @C#C: Feature engineering for 33 features
2020-02-26 12:57:14,102 Joined 33 features into (80542, 33) shape
2020-02-26 12:57:14,102 @FV#C: Feature engineering for 33 features
2020-02-26 12:57:14,451 Joined 33 features into (53497, 33) shape
2020-02-26 12:57:14,452 @TY#C: Feature engineering for 33 features
2020-02-26 12:57:14,802 Joined 33 features into (55083, 33) shape
2020-02-26 12:57:14,802 @TU#C: Feature engineering for 33 features
2020-

2020-02-26 12:57:24,366 GAS#C: Get bins and feature imps
2020-02-26 12:57:24,379 Dropped label 0.0 0.006816632583503749
2020-02-26 12:57:24,381 LF#C: Get bins and feature imps
2020-02-26 12:57:24,400 Dropped label 0.0 0.00216618170061429
2020-02-26 12:57:24,403 LG#C: Get bins and feature imps
2020-02-26 12:57:24,418 Dropped label 0.0 0.0049924891755765664
2020-02-26 12:57:24,420 QCL#C: Get bins and feature imps
2020-02-26 12:57:24,440 Dropped label 0.0 0.0016102001248726628
2020-02-26 12:57:24,442 QHO#C: Get bins and feature imps
2020-02-26 12:57:24,455 Dropped label 0.0 0.00012212994626282364
2020-02-26 12:57:24,457 QNG#C: Get bins and feature imps
2020-02-26 12:57:24,482 Dropped label 0.0 0.0021761725347538004
2020-02-26 12:57:24,485 QPA#C: Get bins and feature imps
2020-02-26 12:57:24,500 Dropped label 0.0 0.0010075566750629723
2020-02-26 12:57:24,502 QPL#C: Get bins and feature imps
2020-02-26 12:57:24,518 Dropped label 0.0 0.0018970736629667004
2020-02-26 12:57:24,521 @QG#C: Get b

C:\Users\doda\pr\fincl\backend\mlbt\reporting.py:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  events["y_pred_proba"] = y_pred_proba
C:\Users\doda\pr\fincl\backend\mlbt\reporting.py:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  events["y_pred"] = y_pred
2020-02-26 13:15:30,530 Run finished: Feature Importance OOS Score: 0.4999780803504733
2020-02-26 13:15:30,531 F1 Score: 0.5119869673888312
2020-02-26 13:15:30,769 config: {'DATA_DIR': Path('C:\\Users\\doda/Dropbox/algotrading/data'), 'F_PAYLOAD_DIR': Path('C:\\Users\\doda/pr/fincl/f

2020-02-26 13:15:34,199 Joined 33 features into (61176, 33) shape
2020-02-26 13:15:34,199 @EMD#C: Feature engineering for 33 features
2020-02-26 13:15:34,507 Joined 33 features into (67589, 33) shape
2020-02-26 13:15:34,508 @NKD#C: Feature engineering for 33 features
2020-02-26 13:15:34,816 Joined 33 features into (73050, 33) shape
2020-02-26 13:15:34,817 @NE#C: Feature engineering for 33 features
2020-02-26 13:15:35,118 Joined 33 features into (45459, 33) shape
2020-02-26 13:15:35,119 @RP#C: Feature engineering for 33 features
2020-02-26 13:15:35,416 Joined 33 features into (33820, 33) shape
2020-02-26 13:15:35,416 BD#C: Feature engineering for 33 features
2020-02-26 13:15:35,722 Joined 33 features into (68392, 33) shape
2020-02-26 13:15:35,723 EZ#C: Feature engineering for 33 features
2020-02-26 13:15:36,044 Joined 33 features into (110422, 33) shape
2020-02-26 13:15:36,045 XG#C: Feature engineering for 33 features
2020-02-26 13:15:36,355 Joined 33 features into (71217, 33) shape
202

...

2020-02-26 13:24:46,994 Dropped label 0.0 0.006736860890514857
2020-02-26 13:24:46,998 @C#C: Get bins and feature imps
2020-02-26 13:24:47,018 Dropped label 0.0 0.010853518508393023
2020-02-26 13:24:47,021 @FV#C: Get bins and feature imps
2020-02-26 13:24:47,038 Dropped label 0.0 0.008400619851561862
2020-02-26 13:24:47,041 @TY#C: Get bins and feature imps
2020-02-26 13:24:47,057 Dropped label 0.0 0.011428571428571429
2020-02-26 13:24:47,060 @TU#C: Get bins and feature imps
2020-02-26 13:24:47,075 Dropped label 0.0 0.018508061986322856
2020-02-26 13:24:47,078 @UB#C: Get bins and feature imps
2020-02-26 13:24:47,092 Dropped label 0.0 0.0059137365816031365
2020-02-26 13:24:47,095 @O#C: Get bins and feature imps
2020-02-26 13:24:47,112 Dropped label 0.0 0.011136493432324385
2020-02-26 13:24:47,115 @SM#C: Get bins and feature imps
2020-02-26 13:24:47,130 Dropped label 0.0 0.0049882352941176475
2020-02-26 13:24:47,133 @S#C: Get bins and feature imps
2020-02-26 13:24:47,152 Dropped label 0.0

2020-02-26 13:25:09,428 Getting model lgbm
2020-02-26 13:25:09,429 hyperparam search n_iter=25 for lgbm on num_threads=32 and n_jobs=-1
C:\Users\doda\Anaconda3\envs\fincl2\lib\site-packages\sklearn\model_selection\_search.py:823: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  "removed in 0.24.", FutureWarning
2020-02-26 13:26:02,179 Using hypers {'num_leaves': 26, 'n_estimators': 50, 'max_depth': 1, 'learning_rate': 0.03, 'colsample_bytree': 0.9999999999999998}
2020-02-26 13:26:02,180 Getting reports for LGBMClassifier
2020-02-26 13:26:02,366 Running validation for LGBMClassifier: round 1/5
2020-02-26 13:26:03,431 Running validation for LGBMClassifier: round 2/5
2020-02-26 13:26:04,685 Running validation for LGBMClassifier: round 3/5
2020-02-26 13:26:06,147 Running validation for LGBMClassifier: round 4/5
2020-02-26 13:26:07,793 Running validation for LGBMClassifier: round 5/5
C:\Users\doda\pr\fincl\backend\mlbt\reporting.py:77: SettingWithCopyW

2020-02-26 13:26:32,599 Joined 33 features into (92410, 33) shape
2020-02-26 13:26:32,600 @CD#C: Feature engineering for 33 features
2020-02-26 13:26:32,915 Joined 33 features into (86716, 33) shape
2020-02-26 13:26:32,916 @ED#C: Feature engineering for 33 features
2020-02-26 13:26:33,212 Joined 33 features into (48965, 33) shape
2020-02-26 13:26:33,213 @ES#C: Feature engineering for 33 features
2020-02-26 13:26:33,512 Joined 33 features into (58387, 33) shape
2020-02-26 13:26:33,513 @NQ#C: Feature engineering for 33 features
2020-02-26 13:26:33,802 Joined 33 features into (31156, 33) shape
2020-02-26 13:26:33,803 @GF#C: Feature engineering for 33 features
2020-02-26 13:26:34,096 Joined 33 features into (41686, 33) shape
2020-02-26 13:26:34,096 @LE#C: Feature engineering for 33 features
2020-02-26 13:26:34,413 Joined 33 features into (58349, 33) shape
2020-02-26 13:26:34,414 @HE#C: Feature engineering for 33 features
2020-02-26 13:26:34,711 Joined 33 features into (61176, 33) shape
202

2020-02-26 13:26:46,556 Dropped label 0.0 0.006736860890514857
2020-02-26 13:26:46,559 @C#C: Get bins and feature imps
2020-02-26 13:26:46,578 Dropped label 0.0 0.010853518508393023
2020-02-26 13:26:46,581 @FV#C: Get bins and feature imps
2020-02-26 13:26:46,597 Dropped label 0.0 0.008400619851561862
2020-02-26 13:26:46,599 @TY#C: Get bins and feature imps
2020-02-26 13:26:46,615 Dropped label 0.0 0.011428571428571429
2020-02-26 13:26:46,618 @TU#C: Get bins and feature imps
2020-02-26 13:26:46,632 Dropped label 0.0 0.018508061986322856
2020-02-26 13:26:46,634 @UB#C: Get bins and feature imps
2020-02-26 13:26:46,648 Dropped label 0.0 0.0059137365816031365
2020-02-26 13:26:46,651 @O#C: Get bins and feature imps
2020-02-26 13:26:46,666 Dropped label 0.0 0.011136493432324385
2020-02-26 13:26:46,669 @SM#C: Get bins and feature imps
2020-02-26 13:26:46,683 Dropped label 0.0 0.0049882352941176475
2020-02-26 13:26:46,685 @S#C: Get bins and feature imps
2020-02-26 13:26:46,704 Dropped label 0.0

2020-02-26 13:27:08,847 Getting model dummy
2020-02-26 13:27:08,848 Using hypers {}
2020-02-26 13:27:08,848 Getting reports for DummyClassifier
2020-02-26 13:27:09,012 Running validation for DummyClassifier: round 1/5
C:\Users\doda\Anaconda3\envs\fincl2\lib\site-packages\sklearn\dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)
2020-02-26 13:27:09,566 Running validation for DummyClassifier: round 2/5
C:\Users\doda\Anaconda3\envs\fincl2\lib\site-packages\sklearn\dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)
2020-02-26 13:27:10,245 Running validation for DummyClassifier: round 3/5
C:\Users\doda\Anaconda3\envs\fincl2\lib\site-packages\sklearn\dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", F

In [9]:
%debug

2020-02-26 13:27:34,603 No traceback has been produced, nothing to debug.
